In [41]:
import os

import numpy as np
import pandas as pd
from pyarrow import parquet
from matplotlib import pyplot as plt
import pymc3 as pm

from pynba.teams import team_id_to_team_abb
from pynba.plot_logos import plot_logos


plt.style.use('blackontrans')
pd.set_option('display.float_format', lambda val: f'{val:.1f}')

In [42]:
DATA_PATH = "/home/matt/Desktop/nba_data/parsed_pbpstats"
LEAGUE = 'nba'
SEASON_TYPE = 'Regular Season'
YEAR = '2016-17'

filters = [[('year', '=', YEAR), ('season_type', '=', SEASON_TYPE), ('league', '=', LEAGUE)]]
possessions = parquet.read_table(DATA_PATH, filters=filters).to_pandas()

In [43]:
def process_possessions(possessions):
    # aggregate possessions per game & offense
    games = possessions.groupby(by=['game_id', 'off_team_id'], as_index=False).agg({
        'def_team_id': 'first',
        'home_team_id': 'first',
        'year': 'first',
        'date': 'first',
        'turnovers': 'sum',
        'threes_attempted': 'sum',
        'threes_made': 'sum',
        'twos_attempted': 'sum',
        'twos_made': 'sum',
        'ft_attempted': 'sum',
        'ft_made': 'sum',
        'off_rebs': 'sum',
        'def_rebs': 'sum',
        'points_scored': 'sum',
        'duration': 'sum',
        'possesion_num': 'count',
        'period': 'max',
    })

    # calculate rate statistics
    games['three_make_rate'] = games['threes_made'] / games['threes_attempted']
    games['two_make_rate'] = games['twos_made'] / games['twos_attempted']
    games['three_attempt_rate'] = games['threes_attempted'] / (games['twos_attempted'] + games['threes_attempted'])
    games['ft_attempt_rate'] = games['ft_attempted'] / games['possesion_num']
    games['ft_make_rate'] = games['ft_made'] / games['ft_attempted']
    games['off_reb_rate'] = games['off_rebs'] / (games['off_rebs'] + games['def_rebs'])
    games['turnover_rate'] = games['turnovers'] / games['possesion_num']
    games['pace'] = games['duration'] / games['possesion_num']
    games['shot_rate'] = (games['threes_attempted'] + games['twos_attempted']) / (games['off_rebs'] + games['possesion_num'])
    games['attempt_rate'] = (games['twos_attempted'] + games['threes_attempted']) / games['possesion_num']
    games['scoring_rate'] = games['points_scored'] / games['possesion_num']

    # estimate scoring rate based on underlying rate statistics
    games['shot_rate_est'] = estimate_shot_rate(
        games['turnover_rate'],
        games['ft_attempt_rate'],
    )
    games['attempt_rate_est'] = estimate_attempt_rate(
        games['shot_rate_est'],
        games['three_attempt_rate'],
        games['three_make_rate'],
        games['two_make_rate'],
        games['off_reb_rate'],
    )
    games['scoring_rate_est'] = calc_scoring_rate(
        games['attempt_rate_est'],
        games['three_make_rate'],
        games['two_make_rate'],
        games['three_attempt_rate'],
        games['ft_attempt_rate'],
        games['ft_make_rate'],
    )

    return games


def estimate_shot_rate(turnover_rate, ft_attempt_rate):
    """
    Estimates shot_rate: shot attempts per potential shot attempt event, i.e.
    possessions plus offensive rebounds. Coefficients are rounded
    from a simple least-squared regression from regular-season data.
    """
    return 1 - 0.85 * turnover_rate - 0.4 * ft_attempt_rate


def estimate_attempt_rate(shot_rate, three_attempt_rate, three_make_rate, two_make_rate, off_reb_rate):
    """
    Estimates attempt_rate (shot attempts per possession) given a shot_rate
    (see above for definition), then taking into account how frequently shots are missed
    and rebounded by the offense.
    """
    no_shot_rate = 1 - shot_rate
    make_rate = three_attempt_rate * three_make_rate + (1 - three_attempt_rate) * two_make_rate
    miss_rate = 1 - make_rate
    def_reb_rate = 1 - off_reb_rate

    restart_possession = shot_rate * miss_rate * off_reb_rate
    last_shot = shot_rate * (make_rate + miss_rate * (def_reb_rate + off_reb_rate * no_shot_rate))
    return last_shot  / (1 - restart_possession) ** 2


def calc_scoring_rate(attempt_rate, three_make_rate, two_make_rate, three_attempt_rate, ft_attempt_rate, ft_make_rate):
    """Calculates points per possession from underlying rate statistics"""
    points_per_attempt = 3 * three_attempt_rate * three_make_rate + 2 * (1 - three_attempt_rate) * two_make_rate
    return attempt_rate * points_per_attempt + ft_attempt_rate * ft_make_rate


games = process_possessions(possessions)

In [51]:
def log5(mu, *args):
    mu0 = mu / (1 - mu)
    p0 = args[0] / (1 - args[0]) / mu0 ** (len(args) - 1)
    for arg in args[1:]:
        p0 *= arg / (1 - arg)
    return p0 / (p0 + 1)


class Inference:
    def __init__(self, games):
        self.games = games
        self.n_teams = self.games['off_team_id'].unique().shape[0]
        self.team_id_to_team_ind = {team_id: team_ind for team_ind, team_id in enumerate(self.games['off_team_id'].unique())}
        self.team_ind_to_team_id = {team_ind: team_id for team_id, team_ind in self.team_id_to_team_ind.items()}

        self.mu_three_make_rate = games['threes_made'].sum() / games['threes_attempted'].sum()
        self.mu_two_make_rate = games['twos_made'].sum() / games['twos_attempted'].sum()
        self.mu_three_attempt_rate = games['threes_attempted'].sum() / (games['twos_attempted'].sum() + games['threes_attempted'].sum())
        self.mu_ft_attempt_rate = games['ft_attempted'].sum() / games['possesion_num'].sum()
        self.mu_ft_make_rate = games['ft_made'].sum() / games['ft_attempted'].sum()
        self.mu_off_reb_rate = games['off_rebs'].sum() / (games['off_rebs'].sum() + games['def_rebs'].sum())
        self.mu_turnover_rate = games['turnovers'].sum() / games['possesion_num'].sum()
        self.mu_pace = games['duration'].sum() / games['possesion_num'].sum()
    
        with pm.Model() as self.model:
            # Off rate priors
            off_three_make_rate = pm.Beta('off_three_make_rate', mu=self.mu_three_make_rate, sigma=0.02, shape=self.n_teams)
            off_two_make_rate = pm.Beta('off_two_make_rate', mu=self.mu_two_make_rate, sigma=0.02, shape=self.n_teams)
            off_three_attempt_rate = pm.Beta('off_three_attempt_rate', mu=self.mu_three_attempt_rate, sigma=0.05, shape=self.n_teams)
            off_off_reb_rate = pm.Beta('off_off_reb_rate', mu=self.mu_off_reb_rate, sigma=0.02, shape=self.n_teams)
            off_turnover_rate = pm.Beta('off_turnover_rate', mu=self.mu_turnover_rate, sigma=0.01, shape=self.n_teams)
            off_ft_attempt_rate = pm.Gamma('off_ft_attempt_rate', mu=self.mu_ft_attempt_rate, sigma=0.02, shape=self.n_teams)
            off_pace = pm.Gamma('off_pace', mu=self.mu_pace, sigma=0.5, shape=self.n_teams)
            off_ft_make_rate = pm.Beta('ft_make_rate', mu=self.mu_ft_make_rate, sigma=0.03, shape=self.n_teams)

            # Def rate priors
            def_three_make_rate = pm.Beta('def_three_make_rate', mu=self.mu_three_make_rate, sigma=0.01, shape=self.n_teams)
            def_two_make_rate = pm.Beta('def_two_make_rate', mu=self.mu_two_make_rate, sigma=0.02, shape=self.n_teams)
            def_three_attempt_rate = pm.Beta('def_three_attempt_rate', mu=self.mu_three_attempt_rate, sigma=0.02, shape=self.n_teams)
            def_off_reb_rate = pm.Beta('def_off_reb_rate', mu=self.mu_off_reb_rate, sigma=0.02, shape=self.n_teams)
            def_turnover_rate = pm.Beta('def_turnover_rate', mu=self.mu_turnover_rate, sigma=0.01, shape=self.n_teams)
            def_ft_attempt_rate = pm.Gamma('def_ft_attempt_rate', mu=self.mu_ft_attempt_rate, sigma=0.02, shape=self.n_teams)
            def_pace = pm.Gamma('def_pace', mu=self.mu_pace, sigma=0.25, shape=self.n_teams)

            # Home rate priors
            home_three_make_rate = pm.Beta('home_three_make_rate', mu=self.mu_three_make_rate + 0.0035, sigma=0.001)
            home_two_make_rate = pm.Beta('home_two_make_rate', mu=self.mu_two_make_rate + 0.0065, sigma=0.002)
            home_off_reb_rate = pm.Beta('home_off_reb_rate', mu=self.mu_off_reb_rate + 0.005, sigma=0.002)
            home_turnover_rate = pm.Beta('home_turnover_rate', mu=self.mu_turnover_rate - 0.0002, sigma=0.001)
            home_ft_attempt_rate = pm.Gamma('home_ft_attempt_rate', mu=self.mu_ft_attempt_rate + 0.005, sigma=0.002)
            home_ft_make_rate = pm.Gamma('home_ft_make_rate', mu=self.mu_ft_make_rate + 0.001, sigma=0.0005)

            # Implied away rate priors (assuming symmetry)
            away_three_make_rate = 2 * self.mu_three_make_rate - home_three_make_rate
            away_two_make_rate = 2 * self.mu_two_make_rate - home_two_make_rate
            away_off_reb_rate = 2 * self.mu_off_reb_rate - home_off_reb_rate
            away_turnover_rate = 2 * self.mu_turnover_rate - home_turnover_rate
            away_ft_attempt_rate = 2 * self.mu_ft_attempt_rate - home_ft_attempt_rate
            away_ft_make_rate = 2 * self.mu_ft_make_rate - home_ft_make_rate

            # Free throw attempt rate and pace distribution priors
            sigma_ft_attempt_rate = pm.Gamma('sigma_ft_attempt_rate', mu=0.05, sigma=0.01)
            sigma_pace = pm.Gamma('sigma_pace', mu=0.8, sigma=0.1)

            # Model rates for each game
            off_index = games['off_team_id'].map(self.team_id_to_team_ind).values
            def_index = games['def_team_id'].map(self.team_id_to_team_ind).values
            home_index = (games['off_team_id'] != games['home_team_id']).values.astype(int)
            games_three_make_rate = log5(
                self.mu_three_make_rate,
                off_three_make_rate[off_index],
                def_three_make_rate[def_index],
                pm.math.stack([home_three_make_rate, away_three_make_rate])[home_index]
            )
            games_two_make_rate = log5(
                self.mu_two_make_rate,
                off_two_make_rate[off_index],
                def_two_make_rate[def_index],
                pm.math.stack([home_two_make_rate, away_two_make_rate])[home_index]
            )
            games_three_attempt_rate = log5(
                self.mu_three_attempt_rate,
                off_three_attempt_rate[off_index],
                def_three_attempt_rate[def_index],
            )
            games_off_reb_rate = log5(
                self.mu_off_reb_rate,
                off_off_reb_rate[off_index],
                def_off_reb_rate[def_index],
                pm.math.stack([home_off_reb_rate, away_off_reb_rate])[home_index]
            )
            games_turnover_rate = log5(
                self.mu_turnover_rate,
                off_turnover_rate[off_index],
                def_turnover_rate[def_index],
                pm.math.stack([home_turnover_rate, away_turnover_rate])[home_index]
            )
            games_ft_attempt_rate = log5(
                self.mu_ft_attempt_rate,
                off_ft_attempt_rate[off_index],
                def_ft_attempt_rate[def_index],
                pm.math.stack([home_ft_attempt_rate, away_ft_attempt_rate])[home_index]
            )
            games_pace = log5(
                self.mu_pace,
                off_pace[off_index],
                def_pace[def_index],
            )
            games_ft_make_rate = log5(
                self.mu_ft_make_rate,
                off_ft_make_rate[off_index],
                pm.math.stack([home_ft_make_rate, away_ft_make_rate])[home_index]
            )

            # Fit model to observations
            pm.Binomial(
                'threes_made',
                n=games['threes_attempted'].values,
                p=games_three_make_rate,
                observed=games['threes_made'].values
            )
            pm.Binomial(
                'twos_made',
                n=games['twos_attempted'].values,
                p=games_two_make_rate,
                observed=games['twos_made'].values
            )
            pm.Binomial(
                'threes_attempted',
                n=games['twos_attempted'].values + games['threes_attempted'].values,
                p=games_three_attempt_rate,
                observed=games['threes_attempted'].values
            )
            pm.Binomial(
                'off_rebs',
                n=games['off_rebs'].values + games['def_rebs'].values,
                p=games_off_reb_rate,
                observed=games['off_rebs'].values
            )
            pm.Binomial(
                'turnovers',
                n=games['possesion_num'].values,
                p=games_turnover_rate,
                observed=games['turnovers'].values
            )
            pm.Normal(
                'ft_attempt_rate',
                mu=games_ft_attempt_rate,
                sigma=sigma_ft_attempt_rate,
                observed=games['ft_attempt_rate'].values
            )
            pm.Gamma(
                'pace',
                mu=games_pace,
                sigma=sigma_pace,
                observed=games['pace'].values
            )
            pm.Binomial(
                'ft_made',
                n=games['ft_attempted'].values,
                p=games_ft_make_rate,
                observed=games['ft_made'].values
            )

    def fit(self):
        with self.model:
            self.trace = pm.sample(20000, init='adapt_diag')

    def traceplot(self):
        pm.traceplot(self.trace);

In [52]:
inference = Inference(games)

In [53]:
inference.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_pace, sigma_ft_attempt_rate, home_ft_make_rate, home_ft_attempt_rate, home_turnover_rate, home_off_reb_rate, home_two_make_rate, home_three_make_rate, def_pace, def_ft_attempt_rate, def_turnover_rate, def_off_reb_rate, def_three_attempt_rate, def_two_make_rate, def_three_make_rate, ft_make_rate, off_pace, off_ft_attempt_rate, off_turnover_rate, off_off_reb_rate, off_three_attempt_rate, off_two_make_rate, off_three_make_rate]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 693 seconds.


In [1]:
off_shot_rate_est = estimate_shot_rate(
    inference.trace['off_turnover_rate'],
    inference.trace['off_ft_attempt_rate'],
)
off_attempt_rate_est = estimate_attempt_rate(
    off_shot_rate_est,
    inference.trace['off_three_attempt_rate'],
    inference.trace['off_three_make_rate'],
    inference.trace['off_two_make_rate'],
    inference.trace['off_off_reb_rate'],
)
off_scoring_rate_est = calc_scoring_rate(
    off_attempt_rate_est,
    inference.trace['off_three_make_rate'],
    inference.trace['off_two_make_rate'],
    inference.trace['off_three_attempt_rate'],
    inference.trace['off_ft_attempt_rate'],
    inference.trace['ft_make_rate'],
)

def_shot_rate_est = estimate_shot_rate(
    inference.trace['def_turnover_rate'],
    inference.trace['def_ft_attempt_rate'],
)
def_attempt_rate_est = estimate_attempt_rate(
    def_shot_rate_est,
    inference.trace['def_three_attempt_rate'],
    inference.trace['def_three_make_rate'],
    inference.trace['def_two_make_rate'],
    inference.trace['def_off_reb_rate'],
)
def_scoring_rate_est = calc_scoring_rate(
    def_attempt_rate_est,
    inference.trace['def_three_make_rate'],
    inference.trace['def_two_make_rate'],
    inference.trace['def_three_attempt_rate'],
    inference.trace['def_ft_attempt_rate'],
    possessions['ft_made'].sum() / possessions['ft_attempted'].sum(),
)

NameError: name 'estimate_shot_rate' is not defined

In [ ]:
results = pd.DataFrame()
results['team'] = [team_id_to_team_abb[inference.team_ind_to_team_id[ind]] for ind in range(inference.trace['off_pace'].shape[1])]
results['off_three_attempt_rate'] = inference.trace['off_three_attempt_rate'].mean(0) * 100
results['off_two_make_rate'] = inference.trace['off_two_make_rate'].mean(0) * 100
results['off_three_make_rate'] = inference.trace['off_three_make_rate'].mean(0) * 100
results['off_reb_rate'] = inference.trace['off_off_reb_rate'].mean(0) * 100
results['off_turnover_rate'] = inference.trace['off_turnover_rate'].mean(0) * 100
results['off_ft_attempt_rate'] = inference.trace['off_ft_attempt_rate'].mean(0) * 100
results['off_ft_make_rate'] = inference.trace['ft_make_rate'].mean(0) * 100
results['def_three_attempt_rate'] = inference.trace['def_three_attempt_rate'].mean(0) * 100
results['def_two_make_rate'] = inference.trace['def_two_make_rate'].mean(0) * 100
results['def_three_make_rate'] = inference.trace['def_three_make_rate'].mean(0) * 100
results['def_reb_rate'] = (1 - inference.trace['def_off_reb_rate'].mean(0)) * 100
results['def_turnover_rate'] = inference.trace['def_turnover_rate'].mean(0) * 100
results['def_ft_attempt_rate'] = inference.trace['def_ft_attempt_rate'].mean(0) * 100
results['off_scoring_rate'] = off_scoring_rate_est.mean(0) * 100
results['def_scoring_rate'] = def_scoring_rate_est.mean(0) * 100
results['net_scoring_rate'] = results['off_scoring_rate'] - results['def_scoring_rate']
results['off_pace'] = (12 * 4 * 60 / inference.trace['off_pace'] / 2).mean(0)
results['def_pace'] = (12 * 4 * 60 / inference.trace['def_pace'] / 2).mean(0)
results['total_pace'] = (12 * 4 * 60 / (inference.trace['off_pace'] + inference.trace['def_pace'])).mean(0)
results['scoring_margin'] = results['net_scoring_rate'] * results['total_pace'] / 100
results = results.sort_values('scoring_margin', ascending=False)
results

In [ ]:
xs = (off_scoring_rate_est - off_scoring_rate_est.mean(1).reshape(-1, 1)).mean(0) * 100
ys = (def_scoring_rate_est.mean(1).reshape(-1, 1) - def_scoring_rate_est).mean(0) * 100
teams = [team_id_to_team_abb[inference.team_ind_to_team_id[ind]] for ind in range(xs.shape[0])]

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
plot_logos(xs, ys, teams, ax, size=30)
ax.set_xlabel('Offensive Efficiency')
ax.set_ylabel('Defensive Efficiency')
ax.set_title(f'{LEAGUE} {YEAR} {SEASON_TYPE}');

In [ ]:
xs = (12 * 4 * 60 / inference.trace['off_pace'] / 2).mean(0)
ys = (12 * 4 * 60 / inference.trace['def_pace'] / 2).mean(0)
teams = [team_id_to_team_abb[inference.team_ind_to_team_id[ind]] for ind in range(xs.shape[0])]

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
plot_logos(xs, ys, teams, ax, size=30)
ax.set_xlabel('Offensive Pace')
ax.set_ylabel('Defensive Pace')
ax.set_title(f'{LEAGUE} {YEAR} {SEASON_TYPE}');